In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [6]:
#q1 jan data

df=pd.read_parquet('/home/dinesh/Programming/mlops/data/yellow_tripdata_2023-03.parquet')
print(f"Number of Columns: {len(df.columns)}")

Number of Columns: 19


In [7]:
#q2
df['duration'] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds() / 60
print(f"Standard deviation of duration: {df['duration'].std()}")


Standard deviation of duration: 43.7945787001033


In [9]:
#q3
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])


n_total = len(df)

df = df[(df['duration'] >= 1) & (df['duration'] <= 60)]

n_filtered = len(df)

# Compute the fraction
fraction = n_filtered / n_total
print(f"Fraction of records kept: {fraction:.3f}")

Fraction of records kept: 1.000


In [11]:
#q4 Load January 2023 Yellow Taxi data
df = pd.read_parquet('/home/dinesh/Programming/mlops/data/yellow_tripdata_2023-03.parquet')

# Filter durations between 1 and 60 mins
df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime).dt.total_seconds() / 60
df = df[(df.duration >= 1) & (df.duration <= 60)]

# Prepare categorical columns
df['PULocationID'] = df['PULocationID'].astype(str)
df['DOLocationID'] = df['DOLocationID'].astype(str)

# Create list of dicts
train_dicts = df[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Vectorize
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

# Check shape
X_train.shape

(3316216, 518)

In [10]:
#q5


# Target variable
y_train = df['duration'].values

# Train model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on train set
y_pred = model.predict(X_train)

# Compute RMSE
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
print(f"RMSE on train: {rmse:.2f}")


RMSE on train: 7.65


In [11]:
#q6
# Load February 2023 data
df_val = pd.read_parquet('yellow_tripdata_2023-02.parquet')

# Compute duration and filter
df_val['duration'] = (df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime).dt.total_seconds() / 60
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

# Prepare features
df_val['PULocationID'] = df_val['PULocationID'].astype(str)
df_val['DOLocationID'] = df_val['DOLocationID'].astype(str)
val_dicts = df_val[['PULocationID', 'DOLocationID']].to_dict(orient='records')

# Transform with fitted DictVectorizer
X_val = dv.transform(val_dicts)

# Predict and compute RMSE
y_val = df_val['duration'].values
y_pred_val = model.predict(X_val)

rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
print(f"RMSE on validation: {rmse_val:.2f}")


RMSE on validation: 7.81
